In [25]:
import pandas as pd

PATH = "../executions/MNQM2022/20220318-20220401.csv"


def get_executions_df(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    df_grouped = df.groupby("date").sum()[["realized_pnl", "px_side"]]

    df_grouped["pnl / px"] = df_grouped["realized_pnl"] / df_grouped["px_side"]
    df_grouped["realized_pnl_cum"] = df_grouped["realized_pnl"].cumsum()
    df_grouped["profit"] = df[df["realized_pnl"] > 0].groupby("date").count()["realized_pnl"]
    df_grouped["loss"] = df[df["realized_pnl"] < 0].groupby("date").count()["realized_pnl"]
    df_grouped.loc["Total"] = df_grouped.sum(numeric_only=True)

    return df_grouped


get_executions_df(PATH)

,realized_pnl,px_side,pnl / px,realized_pnl_cum,profit,loss
date,,,,,,
2022-03-18,2326.020001,320.211667,7.264008,2326.020001,14.0,5.0
2022-03-21,-1456.600001,-306.435834,4.753361,869.420000,12.0,15.0
2022-03-22,943.200000,117.325000,8.039207,1812.620000,3.0,2.0
2022-03-23,-261.500000,-47.263333,5.532830,1551.120000,5.0,9.0
2022-03-24,717.280000,145.170000,4.940966,2268.400000,6.0,3.0
2022-03-25,-69.179999,-76.455000,0.904846,2199.220001,17.0,12.0
2022-03-28,649.840000,53.153333,12.225762,2849.060001,11.0,8.0
2022-03-29,419.240000,47.706667,8.787870,3268.300001,13.0,5.0
2022-03-30,-299.500002,30.120000,-9.943559,2968.799999,21.0,13.0


In [26]:
def get_executions_ungroup(file_path: str):
    df = pd.read_csv(file_path)
    df = df[df["realized_pnl"].notnull()]
    df["date"] = pd.to_datetime(df["time_completed"]).dt.date

    cols = ["quantity", "realized_pnl", "px_side"]

    df_pos = df[df["realized_pnl"] > 0].groupby("date").sum()[["realized_pnl"]]
    df_neg = df[df["realized_pnl"] < 0].groupby("date").sum()[["realized_pnl"]]

    return df.groupby("date").sum()[cols] \
        .join(df_pos, rsuffix="_pos") \
        .join(df_neg, rsuffix="_neg") \
        .join(df.groupby("date").count()[["realized_pnl"]], rsuffix="_count")


get_executions_ungroup(PATH)


,quantity,realized_pnl,px_side,realized_pnl_pos,realized_pnl_neg,realized_pnl_count
date,,,,,,
2022-03-18,57,2326.020001,320.211667,2943.506668,-617.486667,19
2022-03-21,90,-1456.600001,-306.435834,3122.900000,-4579.500001,27
2022-03-22,20,943.200000,117.325000,1809.460000,-866.260000,5
2022-03-23,50,-261.500000,-47.263333,1888.120000,-2149.620000,14
2022-03-24,23,717.280000,145.170000,1050.760000,-333.480000,9
2022-03-25,87,-69.179999,-76.455000,2184.020001,-2253.200000,29
2022-03-28,69,649.840000,53.153333,2311.200000,-1661.360000,19
2022-03-29,59,419.240000,47.706667,1323.340000,-904.100000,18
2022-03-30,150,-299.500002,30.120000,2425.979998,-2725.480000,34
